# 🛠️ Agent Tools with Microsoft Agent Framework (Python)

## 📋 What You'll Learn

This notebook demonstrates how to enhance agents with specialized tools using Azure AI Foundry. You'll see how to add capabilities like file search, code execution, web search, vision processing, and MCP tool integration.

**Tools Covered:**
- 📄 **File Search (RAG)**: Query uploaded documents with vector search
- 💻 **Code Interpreter**: Execute Python code to solve problems
- 🌐 **Web Search (Bing)**: Real-time web information retrieval
- 👁️ **Vision**: Process and analyze images
- 🔌 **MCP Tools**: Connect to external services via Model Context Protocol

## ⚙️ Setup Requirements

### 1. **Docker & Dev Container**
- Install Docker on your machine
- Open this project in the dev container (VS Code will prompt you)

### 2. **Workspace Configuration**
- Open the workspace: `File > Open Workspace from File` → select `workspace.code-workspace`
- This sets up the correct Python environment for each project folder

### 3. **Environment Variables**
1. Copy `.env.example` to `.env` in the `agent-framework-samples` folder:
   ```bash
   cp .env.example .env
   ```
2. Fill in your credentials:
   ```env
   # Azure AI Foundry (Required for all examples)
   AZURE_AI_PROJECT_ENDPOINT=your_project_endpoint
   AZURE_AI_MODEL_DEPLOYMENT_NAME=your_model_deployment
   
   # Bing Search (Required for Web Search example)
   BING_CONNECTION_ID=your_bing_connection_id
   ```

### 4. **Azure AI Foundry Setup**
- Create an Azure AI Foundry project
- Deploy a model (e.g., gpt-4o, gpt-4o-mini)
- For Bing Search: Create a Bing Search connection in your project

## 🚀 What You'll Build

Below you'll find code samples demonstrating:

1. **RAG with File Search**: Upload documents and query them with semantic search
2. **Code Execution**: Let agents write and run Python code (Fibonacci example)
3. **Web Research**: Search the web and get real-time information
4. **Vision Processing**: Analyze images and extract information (furniture example)
5. **MCP Integration**: Two ways to connect external tools via MCP

Each example shows the complete flow: creating the agent, configuring tools, and running queries.

Let's explore agent capabilities! 🌟

In [ ]:
import os

from azure.identity.aio import AzureCliCredential, DefaultAzureCredential
from azure.ai.projects.aio import AIProjectClient
from dotenv import load_dotenv
import base64

from agent_framework.azure import AzureAIAgentClient
from azure.ai.agents.models import (
    FilePurpose,
    FileSearchTool, 
    CodeInterpreterTool, 
    BingGroundingTool)
from agent_framework import (
    ChatAgent,
    HostedFileSearchTool,
    HostedVectorStoreContent, 
    HostedCodeInterpreterTool, 
    HostedWebSearchTool,
    ChatMessage,
    DataContent,
    Role, 
    TextContent,
    MCPStreamableHTTPTool)

from azure.ai.projects.aio import AIProjectClient
load_dotenv()

In [ ]:
async def create_vector_store(client: AIProjectClient) -> tuple[str, HostedVectorStoreContent]:
    """Create a vector store with sample documents."""
    file_path = 'files/document.md'
    file = await client.agents.files.upload_and_poll(file_path=file_path, purpose="assistants")
    print(f"Uploaded file, file ID: {file.id}")


    vector_store = await client.agents.vector_stores.create_and_poll(file_ids=[file.id], name="corporate_insurance")

    print(f"Created vector store, ID: {vector_store.id}")


    return file.id, HostedVectorStoreContent(vector_store_id=vector_store.id)

async with (
        DefaultAzureCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as project_client,
    ):
        file_id, vector_store = await create_vector_store(project_client)
        file_search = FileSearchTool(vector_store_ids=[vector_store.vector_store_id])
        
        created_agent = await project_client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            name="PythonRAGAgent",
            instructions="""
                    You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

                    - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
                    "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
                    - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
                    - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
                    - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
                    - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

                    You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
                    """,
            tools = file_search.definitions,
            tool_resources= file_search.resources
        )
        chat_client=AzureAIAgentClient(agent_id=created_agent.id, async_credential = credential)


        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client,
            ) as agent:
                    

            print("Agent created. You can now ask questions about the uploaded document.")

            query = "Can you explain Contoso's travel insurance coverage?"

            async for chunk in agent.run_stream(query, tools=HostedFileSearchTool(inputs=vector_store)):

                if chunk.text:
                    print(chunk.text, end="", flush=True)

In [ ]:
AgentName = "Coding-Agent"
AgentInstructions = "You are an AI assistant that helps people find information."

code = CodeInterpreterTool()

async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):
        created_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            instructions=AgentInstructions,
            name=AgentName,
            tools=code.definitions
        )
        chat_client=AzureAIAgentClient(project_client=client, agent_id=created_agent.id)

        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client,
                tools=HostedCodeInterpreterTool()
            ) as agent:
            message = "Use code to determine the values in the Fibonacci sequence that that are less then the value of 101?"
            # response = agent.run_stream(message)
            first_result = await agent.run(message)
            
            print(f"Agent: {first_result.text}")

In [ ]:
# Bing Grounding Tool Example 

conn_id = os.environ["BING_CONNECTION_ID"]  # Ensure the BING_CONNECTION_NAME environment variable is set

# Initialize the Bing Grounding tool
bing = BingGroundingTool(connection_id=conn_id)
print(bing)

async with (
        DefaultAzureCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):

        created_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            instructions="You are a helpful assistant that can help with web search questions.",
            name="WebResearchAgent",
            tools=bing.definitions
        )
        chat_client=AzureAIAgentClient(project_client=client, agent_id=created_agent.id)

        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client,
                tools=HostedWebSearchTool()
            ) as agent:
            message = "Could you please describe the workshop according to the link https://github.com/SoTrx/GHCAgentWorkshop?"
            # response = agent.run_stream(message)
            first_result = await agent.run(message)
            
            print(f"Agent: {first_result.text}")

In [ ]:
AgentName = "Vision-Agent"
AgentInstructions = "You are my furniture sales consultant, you can find different furniture elements from the pictures and give me a purchase suggestion"

async with (
        DefaultAzureCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):
        created_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            instructions=AgentInstructions,
            name=AgentName
        )
        chat_client=AzureAIAgentClient(project_client=client, agent_id=created_agent.id)

        async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=chat_client
            ) as agent:
    

            image_path = "files/home.png"
            with open(image_path, "rb") as image_file:
                image_b64 = base64.b64encode(image_file.read()).decode()
            image_uri = f"data:image/png;base64,{image_b64}"
            message = ChatMessage(
                    role=Role.USER,
                    contents=[
                        TextContent(text="Please find the relevant furniture according to the image and give the corresponding price for each piece of furniture"),
                        DataContent(uri=image_uri, media_type="image/png")
                    ]
            )
            # response = agent.run_stream(message)
            first_result = await agent.run(message)
            
            print(f"Agent: {first_result.text}")

# MCP Tool calling : Learn Microsoft MCP Tool

In [ ]:
# Calling the MCP Tool from the agent's context (configuration time) 

async with (
    DefaultAzureCredential() as credential,
    AzureAIAgentClient(async_credential=credential).create_agent(
        name="DocsAgent",
        instructions="You are a helpful assistant that can help with microsoft documentation questions.",
        tools=MCPStreamableHTTPTool(  # Tools defined at agent creation
            name="Microsoft Learn MCP",
            url="https://learn.microsoft.com/api/mcp",
        ),
    ) as agent,
):
    query = "What is Microsoft Semantic Kernel?"
    print(f"User: {query}")
    result = await agent.run(query)
    print(f"{agent.name}: {result}\n")

In [ ]:
# Setting the MCP Tool during the runtime agent's execution  

async with (
    DefaultAzureCredential() as credential,
    MCPStreamableHTTPTool(
        name="Microsoft Learn MCP",
        url="https://learn.microsoft.com/api/mcp",
    ) as mcp_server,
    ChatAgent(
        chat_client=AzureAIAgentClient(async_credential=credential),
        name="DocsAgent",
        instructions="You are a helpful assistant that can help with microsoft documentation questions.",
    ) as agent,
):
    query = "What is Microsoft Semantic Kernel?"
    print(f"User: {query}")
    result = await agent.run(query, tools=mcp_server)
    print(f"{agent.name}: {result}\n")